In [29]:
with open('alice_in_wonderland.txt') as al:
    alice = al.read()

alice = alice.lower()
alice = alice.rstrip("\n").split()
alice = alice[20:] 
print(*alice[:20])
print(alice[:20])

alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to
['alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of', 'sitting', 'by', 'her', 'sister', 'on', 'the', 'bank,', 'and', 'of', 'having', 'nothing', 'to']


In [45]:
punc = "?!./,';:()"
alicep = []
segments = []
for token in alice:
    tokenp = token
    for i , char in enumerate(token):
        if char in punc:
            segments.append(char)
            tokenp[i].strip(char)
    segments.append(tokenp[::-1])
print(segments[:100])

['alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of', 'sitting', 'by', 'her', 'sister', 'on', 'the', ',', 'bank,', 'and', 'of', 'having', 'nothing', 'to', ':', 'do:', 'once', 'or', 'twice', 'she', 'had', 'peeped', 'into', 'the', 'book', 'her', 'sister', 'was', ',', 'reading,', 'but', 'it', 'had', 'no', 'pictures', 'or', 'conversations', 'in', ',', 'it,', '`and', 'what', 'is', 'the', 'use', 'of', 'a', "'", ',', "book,'", 'thought', 'alice', '`without', 'pictures', 'or', "'", '?', "conversation?'", 'so', 'she', 'was', 'considering', 'in', 'her', 'own', 'mind', '(', '(as', 'well', 'as', 'she', ',', 'could,', 'for', 'the', 'hot', 'day', 'made', 'her', 'feel', 'very', 'sleepy', 'and', ',', ')', 'stupid),', 'whether', 'the', 'pleasure', 'of', 'making', 'a', 'daisy-chain']


'dne'